# Step 1: Selection of Station Datasets
After selecting stations based on the criterion you have decided, the station dataset files, initially in .txt format, are converted to .parquet format for faster processing compared to .csv. The code below performs this conversion.

In [ ]:
import os
import pandas as pd

# Define the directories
input_dir = ''
output_dir = ''

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#This step could be optional for you depending on the name of your station dataset text file
# Function to extract station number from the filename and remove leading zero
def extract_station_number(filename):
    parts = filename.split('_')
    station_number = parts[2]  # Extracts the station number (3rd part of the filename)
    return station_number.lstrip('0')  # Removes leading zeros (e.g., '049043' -> '49043')

# Column names based on your provided text file
columns = []

# Loop through all files in the input directory
for file_name in os.listdir(input_dir):
    if file_name.endswith('.txt'):
        station_number = extract_station_number(file_name)
        
        # Construct the full path to the text file
        file_path = os.path.join(input_dir, file_name)
        try:
            # Load the text file into a DataFrame
            df = pd.read_csv(file_path, delimiter=',')
        
            # Save the DataFrame to a Parquet file in the output directory
            output_file = os.path.join(output_dir, f"{station_number}.parquet")
            df.to_parquet(output_file, index=False)
        
            print(f"Converted {file_name} to {station_number}.parquet")
        except UnicodeDecodeError:
            print(f"Skipped {file_name} due to encoding errors.")

# Step 2: Applying Quality Control Checks on Rainfall Values
Please refer to the Supplementary Figure S1 for detailed description on the Quality Control Checks

In [ ]:
import os
import pandas as pd

# Directory containing the Parquet files
parquet_dir = ''

# Ensure output directory exists
output_dir = ''
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Initialize counters
total_accepted = 0
total_non_accepted = 0

# Loop through all Parquet files in the directory
for filename in os.listdir(parquet_dir):
    if filename.endswith('.parquet'):
        file_path = os.path.join(parquet_dir, filename)
        
        try:
            # Load the Parquet file into a DataFrame
            df = pd.read_parquet(file_path)

            # Replace empty strings and spaces with NaN
            df.replace({"": pd.NA, " ": pd.NA}, inplace=True)

            # Check if required columns exist
            required_columns = [
                'Quality of precipitation value', 'Year', 'Rainfall',
                'Number of days of rain within the days of accumulation',
                'Accumulated number of days over which the precipitation was measured'
            ]
            if not all(col in df.columns for col in required_columns):
                print(f"Skipping {filename}: Missing required columns.")
                continue

            # Count total rainfall records (non-NaN rainfall)
            total_rainfall_records = df['Rainfall'].notna().sum()

            # Quality Check 1: Rainfall not NaN, Quality not 'Y', consider if Year > 2019, exclude if Quality is NaN
            condition1 = (
                (~df['Rainfall'].isna()) &
                (df['Quality of precipitation value'] != 'Y') &
                (df['Year'] <= 2019) &
                (~df['Quality of precipitation value'].isna())
            )

            # Quality Check 2: Number of days of rain within accumulation must be 1
            condition2 = (
                (df['Number of days of rain within the days of accumulation'] != 1) &
                (~df['Number of days of rain within the days of accumulation'].isna())
            )

            # Quality Check 3: Accumulated number of days must be 1
            condition3 = (
                (df['Accumulated number of days over which the precipitation was measured'] != 1) &
                (~df['Accumulated number of days over which the precipitation was measured'].isna())
            )

            # Identify discrepancies (non-accepted records)
            discrepancies = df[condition1 | condition2 | condition3]

            # Update Rainfall to -99.99 for discrepant rows
            df.loc[discrepancies.index, 'Rainfall'] = -99.99

            # Count accepted and non-accepted for this file
            non_accepted = len(discrepancies)
            accepted = total_rainfall_records - non_accepted  # Accepted = total - non-accepted

            # Update global counters
            total_accepted += accepted
            total_non_accepted += non_accepted

            # Save the updated DataFrame
            output_file = os.path.join(output_dir, filename)
            df.to_parquet(output_file, index=False)

            print(f"Processed {filename}: Accepted = {accepted}, Non-accepted = {non_accepted}")

        except Exception as e:
            print(f"Error processing {filename}: {e}")

# Print total accepted and non-accepted records
print(f"\nTotal rainfall records accepted: {total_accepted}")
print(f"Total rainfall records non-accepted: {total_non_accepted}")

# Step 3A: Calculating 99th percentile threshold map of New South Wales state with the help of AGCD dataset 


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
xr.set_options(keep_attrs=True, display_expand_data=False)
np.set_printoptions(threshold=10, edgeitems=2)

%xmode minimal
%matplotlib inline
%config InlineBackend.figure_format='retina'

from dask.distributed import Client
client = Client(memory_limit=None, threads_per_worker=1, n_workers=10)
Client()

def _preprocess(ds):
    return ds.precip.sel(lat=slice(-38,-27),lon=slice(140,154))

#Opening the file
dr=xr.open_mfdataset('/agcd_v1-0-1_precip_total_r001_daily_*.nc',
                     parallel=True, preprocess=_preprocess)

dr = dr.chunk({'time':-1, 'lat':25, 'lon':25})



def dask_percentile(array: np.ndarray, axis: str, q: float):
    '''
    Applies np.percentile in dask across an axis
    Parameters:
    -----------
    array: the data to apply along
    axis: the dimension to be applied along
    q: the percentile
   
    Returns:
    --------
    qth percentile of array along axis
   
    Example
    -------
    xr.Dataset.data.reduce(xca.dask_percentile,dim='time', q=90)
    '''
    array = array.rechunk({axis: -1})
    return array.map_blocks(
        np.nanpercentile,
        axis=axis,
        q=q,
        dtype=array.dtype,
        drop_axis=axis)

dr_masked=dr.where(dr>=1)

#Calculating 99 percentile
dr_99=dr_masked.reduce(dask_percentile, q=99,dim='time')
dr_99


dr_99 = dr_99.compute()

# Step 3B: Extracting 99th Percentile threshold for each chosen station dataset

In [ ]:
import pandas as pd
import xarray as xr

# Load the Excel file
file_path = ''
sheet_name = ''
# Load the data from the Excel file
stations_df = pd.read_excel(file_path, sheet_name=sheet_name)

# Load the NetCDF file
dr = xr.open_dataset('/rain_perc99.nc')
prcp = dr.precip_99th

# Create a list to store the 99th percentile precipitation values
r1_99_values = []

# Loop through each station in the DataFrame
for index, row in stations_df.iterrows():
    lat = row['Lat']
    lon = row['Lon']
    # Select the nearest value for the given coordinates
    x = prcp.sel(lat=lat, lon=lon, method='nearest')
    # Append the value to the list
    r1_99_values.append(x.values)

# Add the new column to the DataFrame
stations_df['R1_99'] = r1_99_values

# Save the updated DataFrame back to the same Excel sheet without creating a new sheet
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    # Write the updated DataFrame back to the same sheet
    stations_df.to_excel(writer, index=False, sheet_name=sheet_name)

# Step 4A: Code to get a list of dates from the accepted rainfall records for each station where the rainfall exceeds its 99th percentile value

In [ ]:
import os
import pandas as pd

# Directories and file paths
parquet_dir = ''

#This file would contain the information of 99th percentile threshold for each and every chosen station
excel_file = ''

output_dir = ''

# Read the Excel file (Sheet1) to extract station numbers and corresponding R1_99 values
station_data = pd.read_excel(excel_file, sheet_name='Sheet1')
station_data = station_data[['Station Number', 'R1_99']]

# Function to process each Parquet file
def process_csv(parquet_file):
    # Extract the station number from the filename by removing the .csv extension
    filename = os.path.basename(parquet_file)
    station_number_str = filename.replace('.parquet', '')
    
    try:
        station_number = int(station_number_str)
    except ValueError:
        print(f"Filename '{filename}' does not correspond to a valid station number.")
        return

    # Find the R1_99 value for this station number from the Excel data
    station_info = station_data[station_data['Station Number'] == station_number]
    
    if station_info.empty:
        print(f"No R1_99 value found for station {station_number}")
        return
    
    R1_99_value = station_info['R1_99'].values[0]
    
    # Read the CSV file
    df = pd.read_parquet(parquet_file)
    
    # Convert 'Rainfall' column to numeric (in case it contains invalid data)
    df['Rainfall'] = pd.to_numeric(df['Rainfall'], errors='coerce')
    
    # Filter rows where 'Rainfall' exceeds R1_99 value
    filtered_df = df[df['Rainfall'] > R1_99_value]
    
    if not filtered_df.empty:
        # Save the filtered rows to a new CSV file in the output directory
        output_file = os.path.join(output_dir, f"{station_number}.parquet")
        filtered_df.to_parquet(output_file, index=False)
        print(f"Saved filtered data for station {station_number} to {output_file}")
    else:
        print(f"No extreme rainfall days found for station {station_number}")

# Process all CSV files in the specified directory
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for file in os.listdir(csv_dir):
    if file.endswith('.parquet'):
        csv_file_path = os.path.join(csv_dir, file)
        process_csv(csv_file_path)


# Step 4B: Getting the list of Extreme Rainfall Days

In [ ]:
import os
import pandas as pd
from datetime import datetime

# Define the directories containing the parquet files
directory1 = ""

# Initialize dictionaries to count the number of stations recording rainfall on each day
rainfall_count = {}
rainfall_files = {}

# Function to process each directory for a given year
def process_directory(directory, input_year):
    files_with_data_count = 0
    for filename in os.listdir(directory):
        if filename.endswith(".parquet"):
            filepath = os.path.join(directory, filename)
            df = pd.read_parquet(filepath)
            df = pd.read_parquet(filepath)
            df = df[['Year', 'Month', 'Day', 'Rainfall']]
            df_filtered = df[(df['Year'] == input_year) & (df['Rainfall'] > 0)]  # Filter days with rainfall > 0
            if not df_filtered.empty:
                files_with_data_count += 1
            for _, row in df_filtered.iterrows():
                day = int(row['Day'])
                month = int(row['Month'])
                date = datetime(input_year, month, day).strftime("%Y-%m-%d")
                if date not in rainfall_count:
                    rainfall_count[date] = 0
                    rainfall_files[date] = []
                rainfall_count[date] += 1
                rainfall_files[date].append(filename)
    return files_with_data_count

# Iterate over the year range from 1858 to 2024
for year in range(1858, 2025):
    print(f"Processing year: {year}")
    process_directory(directory1, year)

# Create a DataFrame with all dates in the specified range
date_range = pd.date_range(start='1858-01-01', end='2024-12-31')
rainfall_df = pd.DataFrame(date_range, columns=['Date'])

# Include all dates where at least one station recorded rainfall
rainfall_df['Station Count'] = rainfall_df['Date'].apply(lambda x: rainfall_count.get(x.strftime("%Y-%m-%d"), 0))

# Filter dates where at least one station recorded rainfall (Station Count > 0)
valid_dates = rainfall_df[rainfall_df['Station Count'] > 0]

# Convert the 'Date' column to string format to avoid formatting issues in Excel
valid_dates['Date'] = valid_dates['Date'].dt.strftime('%Y-%m-%d')

# File path to save the data
output_file = ""

# Append the data to the existing Excel file without overwriting
try:
    with pd.ExcelWriter(output_file, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
        existing_data = pd.read_excel(output_file)  # Read the existing data
        new_data = pd.concat([existing_data, valid_dates[['Date', 'Station Count']]])  # Concatenate new data
        new_data.to_excel(writer, index=False, sheet_name='Sheet1')  # Write back to the same sheet
except FileNotFoundError:
    # If the file does not exist, create a new one
    valid_dates[['Date', 'Station Count']].to_excel(output_file, index=False)

# Print the dates where at least one station recorded rainfall
print("Dates where at least one station recorded rainfall:")
print(valid_dates[['Date', 'Station Count']])


# Step 5: Determining the Number of Stations Recording Rainfall ≥ 1.00 mm on Extreme Rainfall Days

In [ ]:
import pandas as pd

# File paths
excel_file_path = ''
parquet_file_path = ''

# Read the Excel file
excel_df = pd.read_excel(excel_file_path)

# Read the Parquet file
parquet_df = pd.read_parquet(parquet_file_path)

# Ensure the 'Date' column in both files is in the same format (YYYY-MM-DD)
excel_df['Date'] = pd.to_datetime(excel_df['Date'])
parquet_df['Date'] = pd.to_datetime(parquet_df['Date'])

# Initialize a list to store station counts for each date
station_counts = []

from tqdm import tqdm

# Initialize a list to store station counts for each date
station_counts = []

# Loop over each date in the Excel file and count the number of stations with Rainfall >= 1.00 mm using tqdm for progress tracking
for date in tqdm(excel_df['Date'], desc="Processing Dates"):
    count = parquet_df[(parquet_df['Date'] == date) & (parquet_df['Rainfall'] >= 1.00)].shape[0]
    station_counts.append(count)

# Add the station counts to the Excel dataframe
excel_df['Station Count'] = station_counts

# Display the updated Excel dataframe
excel_df.head()

# Save the updated Excel dataframe with station counts back to the original Excel file
output_file_path = excel_file_path  # Reusing the same path for saving

# Save the updated dataframe back to the Excel file
with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    excel_df.to_excel(writer, index=False)

# Confirm that the file has been saved
output_file_path



# Step 6: Calculating the Active Station Count for Each Year with Extreme Rainfall Days

In [ ]:
import os
import pandas as pd

# Directory containing the station files
directory = ''

# Initialize a dictionary to store the count of valid stations per year
yearly_station_count = {year: {'count': 0, 'filenames': []} for year in range(1858, 2024)}

# Go through each CSV file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".parquet"):
        file_path = os.path.join(directory, filename)
        
        # Read the CSV file
        df = pd.read_parquet(file_path)
        
        # Ensure 'Year' is treated as integer
        df['Year'] = pd.to_numeric(df['Year'], errors='coerce', downcast='integer')
        
        # Filter out rows where Rainfall is blank or negative, and quality of precipitation is 'N' for years > 2019
        if 'Quality of precipitation value' in df.columns:
            valid_rainfall = df[(df['Rainfall'].notnull()) & (df['Rainfall'] != -99.99) & 
                                 ((df['Year'] >= 2024) | (df['Quality of precipitation value'] != 'N'))]
        else:
            # If the column 'Quality_of_Precipitation' does not exist, just filter based on Rainfall
            valid_rainfall = df[(df['Rainfall'].notnull()) & (df['Rainfall'] != -99.99)]
        
        # Group by 'Year' and process each year
        for year, group in valid_rainfall.groupby('Year'):
            if year in yearly_station_count:
                total_days = len(df[df['Year'] == year])
                
                if total_days > 0 and len(group) / total_days > 0.1:
                    yearly_station_count[year]['count'] += 1
                    # Add filename without .csv extension
                    yearly_station_count[year]['filenames'].append(filename[:-8])

# Convert the result dictionary into a DataFrame
data = []
for year, info in yearly_station_count.items():
    if info['count'] > 0:
        data.append([year, info['count'], ',     '.join(info['filenames'])])

df_yearly_count = pd.DataFrame(data, columns=['Year', 'Station_count', 'Filenames'])

# Save the DataFrame to an Excel file
 excel_path = ''
 df_yearly_count.to_excel(excel_path, index=False)


# Step 7: Assigning Wet to Active Station Ratio to each Extreme Rainfall Day

In [ ]:
import pandas as pd

# File paths
excel_file_path = ''

# Read Sheet1 (ERD_dates) to get station counts
sheet1_df = pd.read_excel(excel_file_path, sheet_name='Sheet1')  # Assuming Sheet1 is the first sheet

# Read Sheet2 (Active Station Count)
sheet2_df = pd.read_excel(excel_file_path, sheet_name='Sheet2')  # Assuming Sheet2 is the second sheet

# Ensure the Year column in Sheet2 is in the same format as the date year in Sheet1
sheet1_df['Year'] = pd.to_datetime(sheet1_df['Date']).dt.year

# Merge the two dataframes on the Year
merged_df = pd.merge(sheet1_df, sheet2_df, on='Year', how='left')

# Calculate the Ratio and add it as a new column
merged_df['Ratio'] = merged_df['Station Count'] / merged_df['Active_Station_count']


# Save the updated Excel dataframe with station counts back to the original Excel file
output_file_path = excel_file_path  # Reusing the same path for saving

# Save the updated dataframe back to the Excel file
with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    merged_df.to_excel(writer, index=False)

# Confirm that the file has been saved
output_file_path


# Step 8A: Classification of Widespread vs Isolated Events
Every Extreme Rainfall day based on their wet to active station ratio is either classified into Widespread Day and Isolated Day. This was done manually in the excel file with the help of filter function

# Step 8B: Getting the list of Consecutive Widespread Day and Non Consecutive Widespread Day

In [ ]:
import pandas as pd

# Load the Excel file
file_path =""
xls = pd.ExcelFile(file_path)

# Read the 'Date' column from the specified sheet
df = pd.read_excel(xls, sheet_name='Sheet1')

# Ensure 'Date' is a datetime object
df['Date'] = pd.to_datetime(df['Date'])

# Find non-consecutive dates
non_consecutive_dates = []
consecutive_dates = []

for i in range(1, len(df) - 1):
    if (df['Date'].dt.date[i] - df['Date'].dt.date[i - 1]).days > 1 and \
       (df['Date'].dt.date[i + 1] - df['Date'].dt.date[i]).days > 1:
        non_consecutive_dates.append(df.iloc[i])
    else:
        consecutive_dates.append(df.iloc[i])

# Create DataFrames from the lists
non_consecutive_df = pd.DataFrame(non_consecutive_dates)
consecutive_df = pd.DataFrame(consecutive_dates)

# Format the 'Date' column to YYYY-MM-DD
non_consecutive_df['Date'] = non_consecutive_df['Date'].dt.strftime('%Y-%m-%d')
consecutive_df['Date'] = consecutive_df['Date'].dt.strftime('%Y-%m-%d')

# Write the non-consecutive dates to Sheet2 and consecutive dates to Sheet3
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:
    non_consecutive_df.to_excel(writer, sheet_name='Sheet3', index=False)
    consecutive_df.to_excel(writer, sheet_name='Sheet4', index=False)

print("Non-consecutive dates copied to Sheet2 and consecutive dates copied to Sheet3.")


# Step 8C: Grouping Consecutive Widespread Day into a single unique events

In [ ]:
import pandas as pd

# Load the Excel file and Sheet 2
file_path = ''
sheet2 = pd.read_excel(file_path, sheet_name='')

# Ensure 'Date' column is in 'YYYY-MM-DD' format
sheet2['Date'] = pd.to_datetime(sheet2['Date'], errors='coerce').dt.strftime('%Y-%m-%d')

# Sort the dataframe by Date
sheet2 = sheet2.sort_values(by='Date').reset_index(drop=True)

# Initialize the 'Event' column as empty strings
sheet2['Event'] = ''

# Assign event names
event_count = 1
sheet2.at[0, 'Event'] = f'Event_{event_count}'  # First event

for i in range(1, len(sheet2)):
    # Check if the current date is consecutive to the previous one
    if pd.to_datetime(sheet2['Date'].iloc[i]) - pd.to_datetime(sheet2['Date'].iloc[i-1]) == pd.Timedelta(days=1):
        # Assign same event for consecutive dates
        sheet2.at[i, 'Event'] = f'Event_{event_count}'
    else:
        # New event for non-consecutive dates
        event_count += 1
        sheet2.at[i, 'Event'] = f'Event_{event_count}'

# Save back to the same Excel file, updating only Sheet 2
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    sheet2.to_excel(writer, sheet_name='', index=False)


# Step 9A: Saving Each Unique Widespread Event Rainfall Data Information

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# Paths
#This file contain unique widespread event identified in Step 8C
excel_path = ""
#This would be single combined parquet file containing all accepted rainfall records
station_data = ""
output_dir = ''

# Load event data
events_df = pd.read_excel(excel_path, sheet_name='Widespread_Event_Cluster')
all_station_data = pd.read_parquet(station_data)

# Group data by 'Event' and get all associated dates
grouped_events = events_df.groupby('Event')

# Loop over each event
for event_name, group in tqdm(grouped_events, desc="Processing Events"):
    # Extract dates for the current event
    event_dates = pd.to_datetime(group['Date'])  # Convert 'Date' column to datetime

    # Filter the combined data for the date range
    filtered_data = all_station_data[all_station_data['Date'].isin(event_dates)]
    
    # Store the rainfall data
    rainfall_data = []
    for _, row in filtered_data.iterrows():
        if row['Rainfall'] >= 0:  # Filter out negative rainfall values
            rainfall_data.append([row['Station Number'], row['Date'].strftime('%Y-%m-%d'), row['Rainfall'], row['Lat'], row['Lon']])

    # Convert rainfall data to a DataFrame
    if rainfall_data:
        rainfall_df = pd.DataFrame(rainfall_data, columns=['Station Number', 'Date', 'Rainfall', 'Latitude', 'Longitude'])
        
        # Pivot the DataFrame to have dates as columns
        pivot_df = rainfall_df.pivot_table(index=['Station Number', 'Latitude', 'Longitude'], columns='Date', values='Rainfall', fill_value=0)
        
        # Reset index to make 'Station Number', 'Latitude', 'Longitude' columns again
        pivot_df.reset_index(inplace=True)

        # Save the DataFrame to a Parquet file
        event_filename = f"{event_name}.parquet"
        output_file_path = os.path.join(output_dir, event_filename)
        pivot_df.to_parquet(output_file_path, index=False)

        print(f"Saved data for {event_name} to {output_file_path}")


# Step 9B: Visualizing Rainfall Dot Maps for Each Identified Widespread Event

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Load the parquet file

#Single file containing all of your accepted rainfall records
parquet_file_path = '/combined_parquet.parquet'

#Containing the rainfall information for a particular widespread event
station_df = pd.read_parquet(parquet_file_path)

#Single file containing all of accepted extreme rainfall records 
extreme_parquet_file_path = '/combined_extreme_parquet.parquet'

extreme_station_df = pd.read_parquet(extreme_parquet_file_path)

# Load the GeoJSON file for the map
geojson_file_path = ''
gdf_map = gpd.read_file(geojson_file_path)

# Separate geometries (polygons and lines)
gdf_polygons = gdf_map[gdf_map.geometry.type == 'Polygon']
gdf_lines = gdf_map[gdf_map.geometry.type == 'LineString']

# Create a custom colormap from white (0 mm), blue (medium), to red (>100 mm)
colors = [(1, 1, 1), (0, 0, 1), (1, 0, 0)]  # white -> blue -> red
rainfall_cmap = LinearSegmentedColormap.from_list("WhiteBlueRed", colors)

# User input prompt for specific date
specific_date_str = input("Enter the date (YYYY-MM-DD) for which you want to generate the Rainfall dot map: ")
specific_date = pd.to_datetime(specific_date_str)

# Filter station data for the specific day
daily_station_data = station_df[
    (station_df['Date'] == specific_date) &
    (station_df['Rainfall'] >= 0.00)  # Include all rainfall values
]

# Filter extreme station data for the same specific day
extreme_station_data = extreme_station_df[
    (extreme_station_df['Date'] == specific_date) &
    (extreme_station_df['Rainfall'] >= 0.00)  # Include all rainfall values
]

# Check if there is any data to plot
if not daily_station_data.empty:
    # Plot the base map with polygons and lines
    fig, ax = plt.subplots(figsize=(20, 5))
    gdf_polygons.plot(ax=ax, color='grey', edgecolor='none', alpha=0.5)
    gdf_lines.plot(ax=ax, color='black', alpha=1)

    # Plot the station markers for general rainfall
    scatter = ax.scatter(
        daily_station_data['Lon'], 
        daily_station_data['Lat'], 
        c=daily_station_data['Rainfall'], 
        cmap=rainfall_cmap,  # Custom colormap
        vmin=0, vmax=100,  # Rainfall from 0 to 100+
        s=50, linewidth=0.5, zorder=2, edgecolor='k'
    )

    # Highlight the extreme stations with a thicker black border
    ax.scatter(
        extreme_station_data['Lon'], 
        extreme_station_data['Lat'], 
        facecolor='none',  # Keep facecolor empty to avoid filling
        edgecolor='black',  # Black border for extreme rainfall
        s=100, linewidth=2, zorder=4  # Thicker black edge with larger size
    )

    # Add color bar for rainfall
    cbar = plt.colorbar(scatter, ax=ax,pad=0.02)
    cbar.set_label('Rainfall (mm)',fontsize=15)
    
    # Title for the plot
    plt.title(f"Rainfall on {specific_date.strftime('%Y-%m-%d')}", fontsize=20)
    # Add labels and title
    plt.xlabel('Longitude', fontsize=15)
    plt.ylabel('Latitude', fontsize=15)

    

    # Save the image
    output_dir = ''
    output_file = f"{output_dir}/1_Rainfall_Extreme_{specific_date.strftime('%Y_%m_%d')}.png"
    plt.savefig(output_file, dpi=300)
    plt.show()
else:
    print(f"No rainfall data available for {specific_date.strftime('%Y-%m-%d')}")


# Step 10: Applying Adaptive Kernel Density Estimation

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KernelDensity
from scipy.ndimage import label, center_of_mass
import os
from tqdm import tqdm
import pyarrow.parquet as pq
from joblib import Parallel, delayed
import logging
import gc


#Directory containing the rainfall information for each unique widespread event
parquet_dir = ''

#File for NSW Map with GDR boundaries highlighted
geojson_file_path = ''

#Output directory to visually see the Extreme Rainfall patterns identified for Each Unique Widespread Event
output_dir = ''

#File for saving the lat and lon coordinates for every identified highest KDE density peak
csv_output_file = ''

# Read the GeoJSON file
gdf_map = gpd.read_file(geojson_file_path)

# Get the bounding box of the map geometries
min_x, min_y, max_x, max_y = gdf_map.total_bounds

# Get a sorted list of Parquet files in the directory
parquet_files = sorted([f for f in os.listdir(parquet_dir) if f.endswith(".parquet")])

# Filter files for events 1 to 3194
event_files = [
    f for f in parquet_files 
    if f.startswith("Event_") and f[6:-8].isdigit() and 1895 <= int(f[6:-8]) <= 2000
]

# Prepare DataFrame to store joined line string coordinates
linestring_df = pd.DataFrame(columns=['Event', 'Longitude', 'Latitude', 'Date'])

# Initialize logging
logging.basicConfig(filename='event_processing.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

def process_event_file(file):
    try:
        # Check if the image for the event is already generated
        output_file = os.path.join(output_dir, f'{file[:-8]}.png')
        if os.path.exists(output_file):
            return f"Skipped {file}, image already exists."

        parquet_file_path = os.path.join(parquet_dir, file)
        df = pq.read_table(parquet_file_path).to_pandas()

        # Replace blank values and -99.9 with NaN
        df.replace([-99.9, '', None], np.nan, inplace=True)

        # Convert columns to numeric
        for col in df.columns[3:]:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        centroids = []

        # Loop through each date column
        for date in df.columns[3:]:
            valid_data = df[['Latitude', 'Longitude', date]].dropna()
            if valid_data.empty:
                continue

            valid_data = valid_data[valid_data[date] > 0.001]
            if valid_data.empty:
                continue

            latitudes = valid_data['Latitude']
            longitudes = valid_data['Longitude']
            rainfall = valid_data[date]
            coords = np.vstack([latitudes, longitudes]).T

            # Initial KDE fitting using rainfall as weights
            kde = KernelDensity(bandwidth=1.5, kernel='gaussian')
            kde.fit(coords, sample_weight=rainfall)
            log_density = kde.score_samples(coords)
            density = np.exp(log_density) + 1e-10
            bandwidths = 1.0 / np.sqrt(density)

            #Applying Adaptive KDE
            adaptive_kde_vals = np.zeros((100, 100))
            grid_lat, grid_lon = np.mgrid[min_y:max_y:100j, min_x:max_x:100j]
            grid_points = np.vstack([grid_lat.ravel(), grid_lon.ravel()]).T

            for i, bw in enumerate(bandwidths):
                kde_adaptive = KernelDensity(bandwidth=bw, kernel='gaussian')
                kde_adaptive.fit(coords[i:i+1], sample_weight=[rainfall.iloc[i]])
                kde_scores = kde_adaptive.score_samples(grid_points)
                adaptive_kde_vals += np.exp(kde_scores).reshape(100, 100)

            #Masking the points which are outside of the map
            mask = np.zeros(adaptive_kde_vals.shape, dtype=bool)
            for geom in gdf_map.geometry:
                if geom.is_valid:
                    points_in_geom = gpd.points_from_xy(grid_lon.ravel(), grid_lat.ravel())
                    points_gdf = gpd.GeoDataFrame(geometry=points_in_geom, crs=gdf_map.crs)
                    mask |= points_gdf.within(geom).values.reshape(adaptive_kde_vals.shape)

            adaptive_kde_vals[~mask] = np.nan

            #Filtering out the values where KDE values are above 95th percentile threshold 
            kde_95_threshold = np.nanpercentile(adaptive_kde_vals, 95)
            high_density_mask = adaptive_kde_vals >= kde_95_threshold

            
            high_density_labels, num_features = label(high_density_mask)
            centroids_of_areas = center_of_mass(adaptive_kde_vals, high_density_labels, range(1, num_features + 1))

            #Getting the highest peak and its corresponding lat and lon information
            if centroids_of_areas:
                largest_centroid = max(centroids_of_areas, key=lambda c: adaptive_kde_vals[int(c[0]), int(c[1])])
                lat_centroid, lon_centroid = grid_lat[int(largest_centroid[0]), 0], grid_lon[0, int(largest_centroid[1])]
                centroids.append((lat_centroid, lon_centroid, date))

        if centroids:
            fig, ax = plt.subplots(figsize=(10, 10))
            gdf_map[gdf_map.geometry.type == 'Polygon'].plot(ax=ax, color='grey', edgecolor='none', alpha=0.5)
            gdf_map[gdf_map.geometry.type == 'LineString'].plot(ax=ax, color='black', alpha=0.5)

            lons = [lon for lat, lon, date in centroids]
            lats = [lat for lat, lon, date in centroids]
            dates = [date for lat, lon, date in centroids]

            ax.plot(lons, lats, color='red', linestyle='-', marker='o')
            for lon, lat, date in zip(lons, lats, dates):
                ax.annotate(date, (lon, lat), textcoords="offset points", xytext=(0, 10), ha='center', fontsize=8, color='red')

            new_rows = []
            event_name = os.path.splitext(file)[0]
            for lon, lat, date in zip(lons, lats, dates):
                new_rows.append({'Event': event_name, 'Longitude': lon, 'Latitude': lat, 'Date': date})

            new_df = pd.DataFrame(new_rows)

            # Append new data to CSV
            new_df.to_csv(csv_output_file, mode='a', header=False, index=False)

            ax.set_title(f'{event_name}', fontsize=16)
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')

            plt.savefig(output_file, bbox_inches='tight')
            plt.close()

        # Track successful processing
        logging.info(f"Successfully processed {file}")
        gc.collect()  # Manually trigger garbage collection
        return f"Processed {file}"

    except Exception as e:
        logging.error(f"Error processing {file}: {e}")
        return f"Error processing {file}"

# Parallel processing with error handling
results = Parallel(n_jobs=56, timeout=1000000)(
    delayed(process_event_file)(file) for file in tqdm(event_files, desc="Processing Parquet files")
)

# After processing all files
logging.info(f"Processing completed. Line string coordinates saved incrementally to {csv_output_file}")
print(f"Line string coordinates saved incrementally to {csv_output_file}")


# Step 11: Identifying and removing Coastal Widespread Extreme Rainfall Event

In [ ]:
import pandas as pd
import geopandas as gpd

# Load the  coastal highlighted polygons  from the GeoJSON file
highlighted_polygons_path = '/highlighted_polygons.geojson'
highlighted_polygons = gpd.read_file(highlighted_polygons_path)

# Load the file which contain the information about lat lon coordinates for every identified highest KDE peak in each unique widespread
# extreme rainfall event data from the Excel file
joined_linestring_path = ''
joined_linestring_df_sheet2 = pd.read_excel(joined_linestring_path, sheet_name="IS")

# Load all the event data (as a GeoDataFrame)
joined_linestring_df = pd.read_excel(joined_linestring_path)
geometry = gpd.points_from_xy(joined_linestring_df['Lon'], joined_linestring_df['Lat'])
joined_linestring_gdf = gpd.GeoDataFrame(joined_linestring_df, geometry=geometry)

# List to store coastal events
coastal_events = []

# Check each event
for event in joined_linestring_gdf['Event'].unique():
    # Filter data for the current event
    event_data = joined_linestring_gdf[joined_linestring_gdf['Event'] == event]
    
    # Check if all points are within any highlighted polygons
    all_within = event_data.geometry.apply(lambda point: highlighted_polygons.contains(point).any()).all()
    
    # If all points are within the polygons, it's a coastal event
    if all_within:
        print(f"{event} is a Coastal Event")
        coastal_events.append(event)

# Remove rows in Sheet2 that belong to coastal events
updated_df_sheet2 = joined_linestring_df_sheet2[~joined_linestring_df_sheet2['Event'].isin(coastal_events)]

# Separate the coastal events data to save in "CS" sheet
coastal_events_df = joined_linestring_df_sheet2[joined_linestring_df_sheet2['Event'].isin(coastal_events)]

# Save the updated DataFrame and coastal events DataFrame back to the Excel file
with pd.ExcelWriter(joined_linestring_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    updated_df_sheet2.to_excel(writer, sheet_name="IS", index=False)  # Update Sheet2
    coastal_events_df.to_excel(writer, sheet_name="ICS", index=False)  # Save coastal events in "CS"

print(f"Removed rows corresponding to the following coastal events from Sheet2 and saved them in the 'CS' sheet: {coastal_events}")


# Step 12: Widespread Event Extreme Rainfall Pattern

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString

# File paths containing the information about lat lon coordinates for every identified highest KDE peak in each unique widespread extreme rainfall event data from the Excel file
excel_file_path = ''
geojson_file_path = ''

# Read the Excel file
df = pd.read_excel(excel_file_path,sheet_name='')

# Filter for Event_X
event_1_data = df[df['Event'] == '']

# Create GeoDataFrame for centroids
geometry = [Point(lon, lat) for lon, lat in zip(event_1_data['Lon'], event_1_data['Lat'])]
gdf_event_1 = gpd.GeoDataFrame(event_1_data, geometry=geometry)

# Calculate the centroid of the points
centroid = gdf_event_1.geometry.unary_union.centroid

# Read the GeoJSON file
gdf_map = gpd.read_file(geojson_file_path)

# Create the plot
fig, ax = plt.subplots(figsize=(10, 6))

# Plot the map
gdf_map[gdf_map.geometry.type == 'Polygon'].plot(ax=ax, color='grey', edgecolor='none', alpha=0.5)
gdf_map[gdf_map.geometry.type == 'LineString'].plot(ax=ax, color='black', alpha=0.5)

# Plot the centroids
gdf_event_1.plot(ax=ax, color='blue', markersize=50, label='Centroids')

# Annotate the centroids with their respective dates
for idx, row in gdf_event_1.iterrows():
    ax.annotate(row['Date'], xy=(row['Lon'], row['Lat']),
                xytext=(3, 3), textcoords="offset points", fontsize=10, color='black')

# Draw lines between the centroids
if len(gdf_event_1) > 1:
    line = LineString(gdf_event_1.geometry.tolist())
    ax.plot(*line.xy, color='blue', linewidth=2)

# Set plot title and labels
ax.set_title('Storm of 1885-01-22 to 1885-01-27', fontsize=20)
ax.set_xlabel('Longitude', fontsize=15)
ax.set_ylabel('Latitude', fontsize=15)
ax.legend()

# Show the plot
plt.show()


# Step 13A: Removing Random 50 stations from the network

In [ ]:
import random
import openpyxl
import os
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.neighbors import KernelDensity
from scipy.ndimage import measurements
from tqdm import tqdm  # Import tqdm for progress bar

# Directory containing the rainfall information for each identified unique widespread extreme rainfall event
base_parquet_dir = ''
geojson_file_path = ''
#File for saving the lat lon coordinates of KDE peak after removing the random 50 stations from the network
output_excel_path = ''

# List of events to analyze
event_list = []

# Read the GeoJSON file
gdf_map = gpd.read_file(geojson_file_path)

# Function to process a single event
def process_event(event_name, gdf_map, output_excel_path):
    parquet_file_path = os.path.join(base_parquet_dir, f"{event_name}.parquet")
    if not os.path.exists(parquet_file_path):
        print(f"File not found: {parquet_file_path}")
        return None

    # Read the Parquet file
    df = pd.read_parquet(parquet_file_path)
    df.replace([-99.9, '', None], np.nan, inplace=True)
    for col in df.columns[3:]:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    min_x, min_y, max_x, max_y = gdf_map.total_bounds
    stations_step = 50
    remaining_stations = len(df)
    all_centroid_tracks = []

    for step in range(200):
        if remaining_stations <= 0:
            break

        sampled_df = df.sample(n=remaining_stations, random_state=42)
        centroids = []

        for date in sampled_df.columns[3:]:
            valid_data = sampled_df[['Latitude', 'Longitude', date]].dropna()
            if valid_data.empty:
                continue

            valid_data = valid_data[valid_data[date] > 0.001]
            if valid_data.empty:
                continue

            coords = np.vstack([valid_data['Latitude'], valid_data['Longitude']]).T
            rainfall = valid_data[date]
            kde = KernelDensity(bandwidth=1.5, kernel='gaussian')
            kde.fit(coords, sample_weight=rainfall)
            log_density = kde.score_samples(coords)
            density = np.exp(log_density) + 1e-10

            bandwidths = 1.0 / np.sqrt(density)
            adaptive_kde_vals = np.zeros((100, 100))
            grid_lat, grid_lon = np.mgrid[min_y:max_y:100j, min_x:max_x:100j]
            grid_points = np.vstack([grid_lat.ravel(), grid_lon.ravel()]).T

            for i, bw in enumerate(bandwidths):
                kde_adaptive = KernelDensity(bandwidth=bw, kernel='gaussian')
                kde_adaptive.fit(coords[i:i+1], sample_weight=[rainfall.iloc[i]])
                kde_scores = kde_adaptive.score_samples(grid_points)
                adaptive_kde_vals += np.exp(kde_scores).reshape(100, 100)

            mask = np.zeros(adaptive_kde_vals.shape, dtype=bool)
            for geom in gdf_map.geometry:
                if geom.is_valid:
                    points_in_geom = gpd.points_from_xy(grid_lon.ravel(), grid_lat.ravel())
                    points_gdf = gpd.GeoDataFrame(geometry=points_in_geom, crs=gdf_map.crs)
                    mask |= points_gdf.within(geom).values.reshape(adaptive_kde_vals.shape)

            adaptive_kde_vals[~mask] = np.nan
            kde_95_threshold = np.nanpercentile(adaptive_kde_vals, 95)
            high_density_mask = adaptive_kde_vals >= kde_95_threshold
            high_density_labels, num_features = measurements.label(high_density_mask)
            centroids_of_areas = measurements.center_of_mass(adaptive_kde_vals, high_density_labels, range(1, num_features + 1))

            if centroids_of_areas:
                largest_centroid = max(centroids_of_areas, key=lambda c: adaptive_kde_vals[int(c[0]), int(c[1])])
                lat_centroid, lon_centroid = grid_lat[int(largest_centroid[0]), 0], grid_lon[0, int(largest_centroid[1])]
                centroids.append((lat_centroid, lon_centroid, date))

        all_centroid_tracks.append(centroids)
        remaining_stations -= stations_step

    centroids_data = []
    for step, centroids in enumerate(all_centroid_tracks):
        for lat, lon, date in centroids:
            centroids_data.append({
                'Filename': event_name,
                'Date': date,
                'Step': step * stations_step,
                'Latitude': lat,
                'Longitude': lon
            })

    centroids_df = pd.DataFrame(centroids_data)
    centroids_pivot = centroids_df.pivot(index=['Filename', 'Date'], columns='Step', values=['Latitude', 'Longitude'])
    centroids_pivot.columns = ['_'.join(map(str, col)) for col in centroids_pivot.columns]
    centroids_pivot.reset_index(inplace=True)

    if os.path.exists(output_excel_path):
        existing_df = pd.read_excel(output_excel_path, sheet_name='Centroids')
    else:
        existing_df = pd.DataFrame()

    updated_df = pd.concat([existing_df, centroids_pivot], ignore_index=True)
    with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
        updated_df.to_excel(writer, index=False, sheet_name='Centroids')

    print(f"Centroid data for {event_name} appended to {output_excel_path}")

for event in tqdm(event_list, desc="Processing events"):
    process_event(event, gdf_map, output_excel_path)


# Step 13B: To Plot extreme rainfall trajectories in subplots at each step removal of stations

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

# File paths containing the information about lat lon coordinates for every identified highest KDE peak in each unique widespread extreme rainfall event data from the Excel file
excel_file_path = ''
geojson_file_path = ''
#Directory containing the rainfall information for each identified unique widespread extreme rainfall event
parquet_dir = ''

# Read the Excel file
df = pd.read_excel(excel_file_path)

# Read the GeoJSON file
gdf_map = gpd.read_file(geojson_file_path)

# Ask the user to select an event (for example, "Event_303")
selected_event = input("Enter the event you want to visualize (e.g., 'Event_303'): ")

# Filter the DataFrame based on the selected event
df_selected_event = df[df['Filename'] == selected_event]

# Extract steps from the columns (e.g., 0, 50, 100, etc.)
steps = [int(col.split('_')[-1]) for col in df_selected_event.columns if col.startswith('Latitude_')]

# Find the corresponding parquet file for the selected event
parquet_file_path = os.path.join(parquet_dir, f'{selected_event}.parquet')

# Read the Parquet file
parquet_df = pd.read_parquet(parquet_file_path)
total_stations = len(parquet_df)

# Calculate total rainfall for each station
rainfall_columns = parquet_df.columns[3:]  # Assuming rainfall data starts from the 4th column
total_rainfall = parquet_df[rainfall_columns].sum(axis=1)

# Add total rainfall to the DataFrame
parquet_df['Total_Rainfall'] = total_rainfall

# Number of rows and columns in subplots
n_cols = 4
n_rows = -(-len(steps) // n_cols)  # Ceiling division to calculate rows needed

# Create a color map for rainfall values
cmap = LinearSegmentedColormap.from_list("rainfall_cmap", ["white", "blue", "red"])

# Create the figure and axes
fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 5))

# Flatten axes for easy indexing (handles case when n_cols > 1)
axes = axes.flatten()

for i, step in enumerate(steps):
    ax = axes[i]

    # Extract latitude and longitude columns for the current step
    lat_col = f'Latitude_{step}'
    lon_col = f'Longitude_{step}'

    # Check if the latitude and longitude columns contain valid data (i.e., not NaN)
    if df_selected_event[lat_col].notna().any() and df_selected_event[lon_col].notna().any():
        # Filter data for current step
        step_latitudes = df_selected_event[lat_col]
        step_longitudes = df_selected_event[lon_col]

        # Plot the GeoJSON map
        gdf_map[gdf_map.geometry.type == 'Polygon'].plot(ax=ax, color='grey', edgecolor='none', alpha=0.8)
        gdf_map[gdf_map.geometry.type == 'LineString'].plot(ax=ax, color='black', linestyle='--',alpha=1,label='GDR boundary')

        # Plot red dots and lines for the step
        ax.scatter(step_longitudes, step_latitudes, color='#483D8B',s=80,edgecolor='black')
        ax.plot(step_longitudes, step_latitudes, color='#483D8B', linestyle='-', alpha=0.8,linewidth=4.5)

        # Shuffle the DataFrame for randomness
        shuffled_parquet_df = parquet_df.sample(frac=1, random_state=42).reset_index(drop=True)


        # Filter stations not removed in this step
        remaining_station_count = total_stations - step
        active_stations =  shuffled_parquet_df.iloc[:remaining_station_count]

        # Plot stations with rainfall values
        sc = ax.scatter(
            active_stations['Longitude'], 
            active_stations['Latitude'], 
            c=active_stations['Total_Rainfall'], 
            cmap=cmap, 
            edgecolor='none', 
            s=30,  # Marker size
            alpha=0.3
        )

        # Store the scatter plot for the colorbar
        scatter_for_colorbar = sc
        
       # Add title and labels with remaining stations
        ax.set_title(f'Remaining Stations: {remaining_station_count}', fontsize=15)
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
    else:
        # If no valid data for this step, hide the axis
        ax.axis('off')

# Hide any extra empty subplots
for j in range(len(steps), len(axes)):
    axes[j].axis('off')

# Add a common colorbar next to the bottom row of subplots
if scatter_for_colorbar:
    cbar_ax = fig.add_axes([0.92, 0.5, 0.02, 0.4])  # [left, bottom, width, height]
    cbar = fig.colorbar(scatter_for_colorbar, cax=cbar_ax)
    cbar.set_label('Total Rainfall (mm)', fontsize=12)

# Add a suptitle for the entire figure
fig.suptitle(f"KDE Extreme Rainfall Track with Stepwise Station Removal", fontsize=20, y=0.95)

# Adjust layout
plt.tight_layout(rect=[0, 0, 0.9, 0.95])  # Leave space for suptitle

# Save the plot
output_path = f'/{selected_event}_Centroid_Tracking.png'
plt.savefig(output_path, dpi=300)

# Show the plot
plt.show()


# Final Step 14: Applying the Dynamic Time Warping Statistical Test

In [ ]:
import pandas as pd
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import matplotlib.pyplot as plt
import os
import pyarrow.parquet as pq
from kneed import KneeLocator
from scipy.ndimage import gaussian_filter1d

# File paths
excel_file = '/Users/coolkarni/Documents/Documents/Documents/Master thesis Data 3 /Centroids_each_day.xlsx'
base_parquet_dir = '/Users/coolkarni/Documents/Documents/Documents/Master thesis Data 3 /19th_Century_Event_Files_parquet_network/'

# Load Excel data
df = pd.read_excel(excel_file)

# Group by Filename (unique events)
grouped = df.groupby('Filename')

# Initialize storage for DTW results
dtw_results_all = {}
elbow_points = []

# Process each unique event
for event, data in grouped:
    base_lat = data['Latitude_0'].values
    base_lon = data['Longitude_0'].values
    
    base_track = list(zip(base_lat, base_lon))
    dtw_results = []
    remaining_stations = []
    
    # Get total stations from corresponding parquet file
    parquet_file = os.path.join(base_parquet_dir, f"{event}.parquet")
    if not os.path.exists(parquet_file):
        print(f"Parquet file for event {event} not found.")
        continue
    
    total_stations = pq.read_table(parquet_file).shape[0]
    
    # Compare base track with other tracks
    for col in range(1, len(data.columns) // 2):  # Assuming Latitude_x and Longitude_x pairs
        lat_col = f"Latitude_{col * 50}"
        lon_col = f"Longitude_{col * 50}"
        
        if lat_col not in data.columns or lon_col not in data.columns:
            continue
        
        current_lat = data[lat_col].dropna().values
        current_lon = data[lon_col].dropna().values
        current_track = list(zip(current_lat, current_lon))
        
        # Skip if the current track is empty
        if not current_track:
            continue
    
        # Calculate DTW distance
        try:
            distance, _ = fastdtw(base_track, current_track, dist=euclidean)
            dtw_results.append(distance)
            remaining_stations.append(total_stations - col * 50)
        except Exception as e:
            print(f"Error calculating DTW for {lat_col} and {lon_col}: {e}")
            continue
    
    # Store results for the event if there are valid DTW distances
    if dtw_results:
        dtw_results_all[event] = (remaining_stations, dtw_results)
        
        # Apply knee point detection for individual events
        if len(dtw_results) > 2:
            kl = KneeLocator(remaining_stations, dtw_results, curve='convex', direction='decreasing', online=True)
            elbow_station = kl.knee
            if elbow_station is not None:
                elbow_points.append(elbow_station)

# Calculate the mean of the elbow points from individual events
mean_elbow_point = np.mean(elbow_points) if elbow_points else None
round_mean_elbow_point = round(mean_elbow_point) if mean_elbow_point is not None else None

# Gather DTW distances for each unique remaining station value across all events
dtw_data_by_station = {}
for event, (stations, distances) in dtw_results_all.items():
    for station, distance in zip(stations, distances):
        if station not in dtw_data_by_station:
            dtw_data_by_station[station] = []
        dtw_data_by_station[station].append(distance)

# Sort remaining stations
sorted_stations = sorted(dtw_data_by_station.keys())

# Calculate mean and standard deviation for each remaining station
mean_distances = []
std_distances = []
for station in sorted_stations:
    values = dtw_data_by_station[station]
    mean_distances.append(np.mean(values))
    std_distances.append(np.std(values))

# Apply Gaussian filter to smooth the mean and standard deviation
sigma = 4  # Standard deviation for Gaussian kernel
smoothed_mean_distances = gaussian_filter1d(mean_distances, sigma)
smoothed_std_distances = gaussian_filter1d(std_distances, sigma)

# Find the best elbow point with varying S values
best_knee = None
best_S = None
for S in np.linspace(1, 1, 50):  # Try different S values (adjust range if needed)
    try:
        knee_locator = KneeLocator(sorted_stations, smoothed_mean_distances, curve="convex", direction="decreasing", S=S)
        if knee_locator.knee:
            best_knee = knee_locator.knee
            best_S = S
            break  # Stop at the first valid knee to optimize
    except ValueError:
        continue

print(f"Best S value: {best_S}, Elbow at: {best_knee}")

# Create the combined plot
plt.figure(figsize=(10, 6))

# Plot individual event trajectories in grey with low alpha
for event, (stations, distances) in dtw_results_all.items():
    plt.plot(stations, distances, marker='none', color='grey', alpha=0.09)

# Plot the smoothed mean line in black
plt.plot(sorted_stations, smoothed_mean_distances, color='black', linewidth=2, label='Mean DTW Distance')

# Shade the area representing ±1 smoothed standard deviation
#plt.fill_between(sorted_stations,
                 #smoothed_mean_distances - smoothed_std_distances,
                 #smoothed_mean_distances + smoothed_std_distances,
                 #color='grey', alpha=0.3, label='±1 Std Dev')

# Highlight the best elbow point from smoothed data
if best_knee is not None:
    plt.scatter(best_knee, knee_locator.knee_y, color='red', s=100, label=f'Critical Station Threshold: {best_knee} Station', alpha=1)

# Highlight the mean elbow point from individual events
if mean_elbow_point is not None:
    plt.axvline(x=mean_elbow_point, color='r', linestyle='--', label=f'Minimum Station Threshold: {round_mean_elbow_point} Station')

# Customize y-axis ticks
max_y = (max(smoothed_mean_distances) + max(smoothed_std_distances) if len(smoothed_mean_distances) > 0 and len(std_distances) > 0 else 0)
y_ticks = np.arange(0, max_y + 4, 4)
plt.yticks(y_ticks)

plt.xlabel('Remaining Stations', fontsize=15)
plt.ylabel('DTW Distance', fontsize=15)
plt.title('Trajectory Analysis', fontsize=20)
plt.legend(loc='upper right', fontsize=12)
plt.grid()
plt.tight_layout()
plt.show()